# 1.   Выбор локации для скважины

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.
<br><br>
<br><br>Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*
<br><br>
<br><br>Шаги для выбора локации:
<br><br>
<br><br>- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
<br><br>- Строят модель и оценивают объём запасов;
<br><br>- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
<br><br>- Прибыль равна суммарной прибыли отобранных месторождений.</div>

## 1.1.   Загрузка и подготовка данных

### 1.1.1.   Импорт модулей

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

### 1.1.2.   Чтение и знакомство с данными

In [2]:
data0 = pd.read_csv('/datasets/geo_data_0.csv')
data1 = pd.read_csv('/datasets/geo_data_1.csv')
data2 = pd.read_csv('/datasets/geo_data_2.csv')

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Взглянем на датасеты и их характеристики</div>

In [3]:
display(data0.head(), data0.shape)
display(data1.head(), data1.shape)
display(data2.head(), data2.shape)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


(100000, 5)

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


(100000, 5)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


(100000, 5)

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Здесь все культурно, имеется много данных, проверим на ошибки и соответствие типам данных</div>

In [4]:
print(data0.info(), '\n\n')
print(data1.info(), '\n\n')
print(data2.info(), '\n\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Co

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Здесь так же порядок – данные принадлежат нужным типам, ошибок нигде нет</div>

## 1.2.   Обучение и проверка модели

### 1.2.1.   Создание функции

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Создадим функцию, которая будет автоматически разбивать данные и обучать модель</div>

In [5]:
# функция принимает признаки, целевой признак, размеры выборок 
# и доп. параметр, долгоиграющий – нужен ли принт (в будущем может помешаться)
def linear_auto_model(features, target, print_=True):
    features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                                  target, 
                                                                                  test_size=0.25, 
                                                                                  random_state=0)
    #код ревьюера                                                                           random_state=0)
    print(target_valid.shape)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    if print_:
        print(f'Средний запас по предсказанию: {predicted_valid.mean()}')
        print(f'RMSE модели: {mean_squared_error(predicted_valid, target_valid)**0.5}')
    # так как в будущем нам понадобятся эти модели – вернем их сразу после обучения
    return model, (predicted_valid, target_valid)

### 1.2.2.   Проверка моделей

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Трансформируем признаки</div>

In [6]:
# разделяем данные на выборки
features_train, features_valid, target_train, target_valid = train_test_split(data0[['f0', 'f1', 'f2']], 
                                                                                  data0['product'], 
                                                                                  test_size=0.25, 
                                                                                  random_state=0)

# обучаем Scaler на тренировочной выборке
scaler = StandardScaler().fit(features_train)

data0[['f0', 'f1', 'f2']] = scaler.transform(data0[['f0', 'f1', 'f2']])

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
И так со всеми</div>

In [7]:
features_train, features_valid, target_train, target_valid = train_test_split(data1[['f0', 'f1', 'f2']], 
                                                                                  data1['product'], 
                                                                                  test_size=0.25, 
                                                                                  random_state=0)
scaler = StandardScaler().fit(features_train)

data1[['f0', 'f1', 'f2']] = StandardScaler().fit_transform(data1[['f0', 'f1', 'f2']])

In [8]:
features_train, features_valid, target_train, target_valid = train_test_split(data2[['f0', 'f1', 'f2']], 
                                                                                  data2['product'], 
                                                                                  test_size=0.25, 
                                                                                  random_state=0)
scaler = StandardScaler().fit(features_train)

data2[['f0', 'f1', 'f2']] = StandardScaler().fit_transform(data2[['f0', 'f1', 'f2']])

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Теперь проверим функцию на наших данных</div>

In [9]:
linear_auto_model(data0.drop(['id', 'product'], axis=1), data0['product'])[0]

(25000,)
Средний запас по предсказанию: 92.27144852242301
RMSE модели: 37.48100896950594


LinearRegression()

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Разброс в районе 38 пунктов при среднем в 91 – очень немало, такая модель не может пойти в работу</div>

In [10]:
linear_auto_model(data1.drop(['id', 'product'], axis=1), data1['product'])[0]

(25000,)
Средний запас по предсказанию: 69.15162398290754
RMSE модели: 0.887257305221933


LinearRegression()

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Здесь все намного более удачно - разброс малый, по сравнению с масштабом данных</div>

In [11]:
linear_auto_model(data2.drop(['id', 'product'], axis=1), data2['product'])[0]

(25000,)
Средний запас по предсказанию: 94.70753129105672
RMSE модели: 40.31290686044374


LinearRegression()

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
История циклична, и в третьем регионе разброс по предсказаниям так же высок, как и в первом</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Вторая модель имеет аномально низкий показатель RMSE, по сравнению с остальными двумя параметрами. Стоит проверить ее данные на корреляцию</div>

In [12]:
data1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
3%, 1%, 99% – это говорит о сильной связи третьего параметра с переменной product – отсюда и малый разброс: данные легко предсказать</div>

### 1.2.3.   Вывод

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Самый малый разброс показал второй регион, скорее всего, это связано с высокой корреляцией. Первый и третий регионы имеют очень большой RMSE, вследствие чего таким данным сложно доверять</div>

## 1.3.   Подготовка к расчёту прибыли

### 1.3.1.   Сохранение основных показателей

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Сохраним основные данные для расчета</div>

In [13]:
# бюджет для строительства скважин – 10 млрд
BUDGET = 10*(10**9)
# доход с единицы объема - 450 тыс
PROFIT = 450000
# количество скважин для строительства – 200
BOREHOLES = 200

### 1.3.2.   Расчет необходимого запаса

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Создадим функцию для расчета необходимого запаса в одной скважине для выхода в ноль</div>

In [14]:
def one_borehole_min_value(budget, one_point_profit, borehole_count):
    return budget / one_point_profit / borehole_count

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Посмотрим на результат при наших данных</div>

In [15]:
one_borehole_min_value(BUDGET, PROFIT, BOREHOLES)

111.11111111111111

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
111 единиц сырья – немало, учитывая имеющиеся малые объемы в других скважинах</div>

### 1.3.3.   Сравнение значений

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Теперь посмотрим на средние значения скважин в разных регионах</div>

In [16]:
data0['product'].mean()

92.50000000000001

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
В первом регионе, в среднем, 92 единицы сырья в скважине, что не очень далеко от требуемых значений</div>

In [17]:
data1['product'].mean()

68.82500000000002

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Во втором регионе значительно меньший средний показатель – всего лишь 68 единиц сырья</div>

In [18]:
data2['product'].mean()

95.00000000000004

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
В третьем регионе среднее значение больше всех – 95 единиц сырья в скважине</div>

### 1.3.4.   Функция поиска лучших скважин

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Теперь создадим функцию, рассчитывающую прибыль с 200 самых удачных скважин в регионе</div>

In [19]:
def best_boreholes_profit(region_data, predicted_boreholes_data):
    # сортируем полученные значения по убыванию и берем первые 200 элементов
    sorted_predicted = predicted_boreholes_data.sort_values(ascending=False)[:BOREHOLES]
    return sum(region_data[sorted_predicted.index]) * PROFIT, sorted_predicted

### 1.3.5.   Проверка на данных

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Проверим их на наших регионах, предсказав прибыль с каждого </div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Построим модель на данных из первого региона</div>

In [20]:
model, (predicted0_valid, target0_valid) = linear_auto_model(data0.drop(['id', 'product'], axis=1), 
                                                           data0['product'], 
                                                           print_=False)

(25000,)


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
По очереди для каждого региона рассчитаем 200 лучших скважин и их прибыль</div>

In [21]:
region0_profit = best_boreholes_profit(pd.Series(target0_valid).reset_index(drop=True), pd.Series(predicted0_valid))

In [22]:
model, (predicted1_valid, target1_valid) = linear_auto_model(data1.drop(['id', 'product'], axis=1), 
                                                             data1['product'], 
                                                             print_=False)

(25000,)


In [23]:
region1_profit = best_boreholes_profit(pd.Series(target1_valid).reset_index(drop=True), pd.Series(predicted1_valid))

In [24]:
model, (predicted2_valid, target2_valid) = linear_auto_model(data2.drop(['id', 'product'], axis=1), 
                                                             data2['product'], 
                                                             print_=False)

(25000,)


In [25]:
region2_profit = best_boreholes_profit(pd.Series(target2_valid).reset_index(drop=True), pd.Series(predicted2_valid))

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Проверим прибыль по каждому региону, сравнив с бюджетом на строительство скважин</div>

In [26]:
region0_profit[0] - BUDGET

3363408079.60795

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
В первом регионе с 200 лучших скважин мы получим плюс на 3.3 млрд</div>

In [27]:
region1_profit[0] - BUDGET

2415086696.681551

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Во втором регионе – также в плюс, но уже на 2.4 млрд</div>

In [28]:
region2_profit[0] - BUDGET

2623878683.516424

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Третий регион выглядит чуть более привлекательно – плюс, хоть и всего на 2.6 млрд</div>

### 1.3.6.   Вывод

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
По предсказанным моделью машинного обучения данным, самым выгодным регионом (из имеющихся данных) является третий регион. Однако, все они все еще убыточны</div>

## 1.4.   Расчёт прибыли и рисков

### 1.4.1.   Расчет Bootstrap

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Применим Bootstrap для поиска среднего всего датасета</div>

In [29]:
state = np.random.RandomState()

def bootstrap(data, repeat):
    samples_mean = []
    for i in range(repeat):
        sample_data = data.sample(n=500, replace=True, random_state=state)
        samples_mean.append(best_boreholes_profit(data, sample_data)[0])
    return samples_mean

In [30]:
#код ревьюера
rng = np.random.RandomState()
print(rng.rand(5))
rng = np.random.RandomState()
print(rng.rand(5))

[0.45234238 0.68501098 0.37413058 0.00407884 0.05250469]
[0.19263683 0.75424956 0.49931257 0.51387144 0.82740393]


### 1.4.2.   Применение Bootstrap

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Проверим средние значения bootstrap на каждом регионе</div>

In [31]:
region0_bootstrap = bootstrap(data0['product'], 1000)
print(pd.Series(region0_bootstrap).mean() - BUDGET)

2385709119.823578


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
В первом регионе в среднем мы бы получили прибыль в 2.4 млрд</div>

In [32]:
region1_bootstrap = bootstrap(data1['product'], 1000)
print(pd.Series(region1_bootstrap).mean() - BUDGET)

440478975.5426369


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Во втором регионе уже 0.44, заметно меньше</div>

In [33]:
region2_bootstrap = bootstrap(data2['product'], 1000)
print(pd.Series(region2_bootstrap).mean() - BUDGET)

2634145919.1002655


<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Третий регион близок к первому – 2.6</div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Эти данные выглядят чуть более обнадеживающе, ни один из регионов не оказался убыточным</div>

### 1.4.3.   Доверительный интервал

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Рассчитаем 95% доверительный интервал для каждого такого значения </div>

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Проверим по регионам</div>

In [34]:
pd.Series(region0_bootstrap).quantile(0.975) - pd.Series(region0_bootstrap).quantile(0.025)

766436539.7390442

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
В первом регионе масштаб 95% доверительного интервала в 774 млн</div>

In [35]:
pd.Series(region1_bootstrap).quantile(0.975) - pd.Series(region1_bootstrap).quantile(0.025)

817902925.985405

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
Во втором регионе больше – 789 млн</div>

In [36]:
pd.Series(region2_bootstrap).quantile(0.975) - pd.Series(region2_bootstrap).quantile(0.025)

805562235.0681667

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
В третьем регионе лучший результат - 819 млн</div>

### 1.4.4.   Вывод

<div style="background-color: #fff0e0; padding: 10px; font-family: monospace; font-size: 15px">
По всем данным, которые мы смогли получить – выгоднее всего выглядит третий регион, в котором доверительный интервал на 95% выше остальных, а так же прибыль с работы скважин больше, чем в других регионах</div>